In [6]:
!pip install transformers datasets evaluate peft sentencepiece accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 96.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [55]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, AutoTokenizer
from sklearn.model_selection import train_test_split
import torch
from datasets import Dataset
from torch.utils.data import DataLoader

In [3]:
def parse_data_2014(xml_path):
    container = []  # Initialize Container (List) for Parse Data
    sentences = ET.parse(xml_path).getroot()  # Get Sentence-Level Nodes

    for sentence in sentences:  # Loop Through Sentences
        sentence_id = sentence.attrib["id"]  # Save ID
        sentence_text = sentence.find('text').text  # Save Text
        aspects = sentence.findall('*')  # Get Aspect-Level Nodes

        found_category = False

        for aspect in aspects:  # Loop Through Aspects
            if aspect.tag == "aspectCategories":
                opinions = aspect.findall('*')  # Get Opinion-Level Nodes
                for opinion in opinions:
                    category = opinion.attrib["category"]
                    polarity = opinion.attrib.get("polarity", np.nan)
                    row = {"sentence_id": sentence_id, "sentence": sentence_text, "category": category, "polarity": polarity}
                    container.append(row)
                found_category = True

        if not found_category:
            row = {"sentence_id": sentence_id, "sentence": sentence_text, "category": np.nan, "polarity": np.nan}
            container.append(row)

    return pd.DataFrame(container)

In [47]:
class ABSA_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [58]:
def setup_data(dataset, tokenizer):
  """
function to set up the parsed data which is in pandas dataframe format

  """
  label_map = {'negative': 0, 'neutral': 1, 'positive': 2, 'conflict':3}
  dataset_texts = dataset['sentence']
  dataset_labels = dataset['polarity']
  dataset_labels =  [label_map[label] for label in dataset_labels]

  dataset_encodings = tokenizer(dataset_texts, padding="max_length", truncation=True, max_length=512)
  dataset = ABSA_Dataset(dataset_encodings, dataset_labels)

  return dataset

In [62]:
from sklearn.metrics import f1_score

def evaluate(model, dataloader, device):
    """
    Function to evaluate the model
    Input: the pretrained model, the dataloader, and the device
    """
    model.eval()
    eval_loss = 0
    correct_predictions = 0
    total_predictions = 0
    all_labels = []
    all_predicted_labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            eval_loss += loss.item()
            _, predicted_labels = torch.max(logits, dim=1)
            correct_predictions += (predicted_labels == labels).sum().item()
            total_predictions += labels.size(0)

            # Collecting all true and predicted labels for F1 score calculation
            all_labels.extend(labels.cpu().numpy())
            all_predicted_labels.extend(predicted_labels.cpu().numpy())

    eval_loss /= len(dataloader)
    accuracy = correct_predictions / total_predictions
    f1 = f1_score(all_labels, all_predicted_labels, average='macro')

    return eval_loss, accuracy, f1


In [52]:
#loading the dataset

xml_file = 'Restaurants_Train.xml'
parsed_data = parse_data_2014(xml_file)
parsed_data

,sentence_id,sentence,category,polarity
0,3121,But the staff was so horrible to us.,service,negative
1,2777,"To be completely fair, the only redeeming fact...",food,positive
2,2777,"To be completely fair, the only redeeming fact...",anecdotes/miscellaneous,negative
3,1634,"The food is uniformly exceptional, with a very...",food,positive
4,2534,Where Gabriela personaly greets you and recomm...,service,positive
...,...,...,...,...
3709,1063,But that is highly forgivable.,anecdotes/miscellaneous,positive
3710,777,"From the appetizers we ate, the dim sum and ot...",food,positive
3711,875,"When we arrived at 6:00 PM, the restaurant was...",anecdotes/miscellaneous,neutral
3712,671,Each table has a pot of boiling water sunken i...,food,neutral


In [56]:
model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/test_trainer/checkpoint1/checkpoint-93', num_labels=4)

In [64]:

#setting up the dataset
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

df = parsed_data

df = Dataset.from_pandas(df)


eval_df = df.select([i for i in range(1000,2000)])

eval_dataset = setup_data(eval_df, tokenizer)


eval_loader = DataLoader(eval_dataset, batch_size=8)




# evaluating the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

eval_loss, accuracy, f1_score = evaluate(model, eval_loader, device)
print(f'Evaluation Loss: {eval_loss}, Accuracy: {accuracy}, F1_score: {f1_score}')

Evaluation Loss: 0.8310921156690234, Accuracy: 0.706, F1_score: 0.35933229545061407
